# Train YOLO model and save the weights

In [1]:
import os
import shutil
import torch
from ultralytics import YOLO

from src.config import ROOT_DIR, YOLO_WEIGHTS_PATH

### Download dataset

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download kpoviesistphane/tomato-leaf-disease-detection --unzip

In [ ]:
def change_dataset_config_pathnames(
    config_file_path: str,
    train_images_path: str,
    valid_images_path: str,
    test_images_path: str, 
) -> None:
    """
    Change the pathnames of train, validation, and test images folders in the dataset config file.

    Args:
        config_file_path (str): The path to the dataset config file.
        train_images_path (str): The path to the training images.
        valid_images_path (str): The path to the validation images.
        test_images_path (str): The path to the test
    
    Returns:
        None
    """

    with open(config_file_path, 'r') as f:
        content = f.readlines()

    for i, line in enumerate(content):
        if line.startswith('train:'):
            content[i] = f'train: {train_images_path}\n'
        elif line.startswith('val:'):
            content[i] = f'val: {valid_images_path}\n'
        elif line.startswith('test:'):
            content[i] = f'test: {test_images_path}\n'

    with open(config_file_path, 'w') as f:
        f.writelines(content)

In [ ]:
dataset_path = os.path.join(ROOT_DIR, "Tomato-Leaf-Disease-63")
dataset_config_file_path = os.path.join(dataset_path, "data.yaml")
train_images_path = os.path.join(dataset_path, "train", "images")
valid_images_path = os.path.join(dataset_path, "valid", "images")
test_images_path = os.path.join(dataset_path, "test", "images")
training_results_path = os.path.join(ROOT_DIR, "runs")

In [ ]:
change_dataset_config_pathnames(
    dataset_config_file_path,
    train_images_path,
    valid_images_path,
    test_images_path
)

Remove previous training results

In [ ]:
if os.path.exists(training_results_path):
    shutil.rmtree(training_results_path)

### Train model

In [ ]:
model = YOLO('yolov8x.pt')

model.info()

In [ ]:
if torch.cuda.is_available():
    print("CUDA is availible")
    model.to('cuda')
else:
    print("CUDA is not availible")

In [ ]:
# Training the model
results = model.train(
    data=dataset_config_file_path,
    epochs=100,
    imgsz=512,
    batch=8,
    workers=4,
    device=0 if torch.cuda.is_available() else 'cpu',
)

Move the best YOLO weights to the root directory

In [ ]:
trained_weights_path = os.path.join(training_results_path, "detect", "train", "weights")

shutil.move(os.path.join(trained_weights_path, "best.pt"), YOLO_WEIGHTS_PATH)
shutil.rmtree(trained_weights_path)